In [2]:
import pandas as pd
import numpy as np
import re
import regex
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from Carga_dataset import *

In [21]:
#Convertimos la columna en string para poder trabajar con el texto
df['Descripcion limpia']= df['descripcion'].astype(str)
df['Descripcion limpia']

id_producto
2792-88-LE21-1        CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...
1704-52-LP21-1        SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...
1003-20-L121-1                                    LEÑA SECA CERTIFICADA
4236-21-L121-1                             MARCADOR BLANCO PARA METALES
4236-21-L121-2                                                  PLOMADA
                                            ...                        
1058141-35-LQ21-1              ALTEPLASA FA 50 MG CAJA X 2 FA [216P008]
1058141-35-LQ21-4           TENECTEPLASA FA 10.000 UI (50 MG) [216P185]
1173455-12-LE21-41               manguera transparente para nivel (mts)
1057461-64-LE21-17                        CINTA ADHESIVA 18 MM X 30 MT 
4309-381-LQ21-1       10-100-214-208-01 Olanzapina tableta liofiliza...
Name: Descripcion limpia, Length: 54849, dtype: object

In [5]:
#Removemos la palabra "no" de nuestro stopword ya que nos puede ser útil
sw = stopwords.words('spanish')
sw.remove("no")

In [19]:
#Definimos nuestra función para quitar las tildes
def sin_tildes(s):
    tildes = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for origen, destino in tildes:
        s = s.replace(origen, destino)
    return s




#Definimos una función para el preprocesamiento de texto

def texto_limpio(texto):
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W\d_]+", " ",texto) # remover caract especiales y números
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if len(palabra) > 3] # eliminar palabras con menos de 3 letras
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = " ".join(texto)
    return texto

In [22]:
df['Descripcion limpia'] = df['Descripcion limpia'].apply(lambda texto: texto_limpio(texto)) #Aplicamos la función texto_limpio para limpiar las descripciones
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...,cam retinal no midriat especif bas tecnic
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,1,0,0,0,0,0,0,servicio mudanza enseres propios hospital mart...,servici mudanz enser propi uso hospital san ma...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,leña seca certificada,leñ sec certific
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,marcador blanco metales,marcador blanc metal
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,plomada,plom


In [38]:
stemmer=SnowballStemmer("spanish")
#Obtención de texto raíz limpio
def texto_raiz(texto):    
    texto = texto.lower() # convertir en minúsculas
    texto = re.sub(r"[\W\d_]+", " ",texto) # remover caract especiales y números
    texto = sin_tildes(texto) # remove tildes
    texto = texto.split() # tokenizar
    texto = [palabra for palabra in texto if len(palabra) > 3] # eliminar palabras con menos de 3 letras
    texto = [palabra for palabra in texto if palabra not in sw] # stopwords
    texto = [stemmer.stem(palabra) for palabra in texto] #stem
    texto = " ".join(texto)
    
    return texto

In [39]:
df['Descripcion raiz limpia']= df['Descripcion limpia'].apply(lambda texto: texto_raiz(texto)) #Aplicamos la función texto_raiz que nos convierte las palabras en sus raíces
df.head()

,id_licitacion,id_producto,Rubro2,Rubro3,nombre_producto,descripcion,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,...,Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion limpia,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,2792-88-LE21,2792-88-LE21-1,Productos quirúrgicos,"Equipo quirúrgico, accesorios y productos rela...",Equipo quirúrgico de fragmatome retinal vítreo...,CÁMARA RETINAL NO MIDRIÁTICA - ESPECIFICACIONE...,0,0,0,0,...,0,0,0,0,0,0,0,0,camara retinal midriatica especificaciones bas...,cam retinal midriat especif bas tecnic
1704-52-LP21-1,1704-52-LP21,1704-52-LP21-1,Transporte de correo y carga,Transporte de carga por carretera,Servicios de transporte local en camión,SERVICIO DE MUDANZA DE ENSERES PROPIOS Y EN US...,0,0,0,0,...,0,1,0,0,0,0,0,0,servicio mudanza enseres propios hospital mart...,servici mudanz enser propi hospital martin qui...
1003-20-L121-1,1003-20-L121,1003-20-L121-1,Chatarra y desechos no metálicos,Desechos no metálicos,Desperdicios o restos de madera,LEÑA SECA CERTIFICADA,0,0,0,0,...,0,0,0,0,0,0,0,0,leña seca certificada,leñ sec certific
4236-21-L121-1,4236-21-L121,4236-21-L121-1,Suministros de oficina,Instrumentos de escritura,Marcadores,MARCADOR BLANCO PARA METALES,0,0,0,0,...,0,0,0,0,0,0,0,0,marcador blanco metales,marcador blanc metal
4236-21-L121-2,4236-21-L121,4236-21-L121-2,Herramientas manuales,Herramientas de medición y trazado,Péndulos,PLOMADA,0,0,0,0,...,0,0,0,0,0,0,0,0,plomada,plom


In [53]:
#Creamos un nuevo df con los datos deseados para la matriz TD-IDF
df1=df.iloc[:, 6:]
df1.drop(['Descripcion limpia'], axis=1)


,label,Rubro1_Artículos de electrónica,Rubro1_Artículos de fabricación y producción,Rubro1_Artículos eléctricos y de iluminación,"Rubro1_Artículos para estructuras, obras y construcciones",Rubro1_Artículos para plantas y animales,"Rubro1_Combustibles, lubricantes y anticorrosivos",Rubro1_Consultoria,"Rubro1_Educación, formación, entrenamiento y capacitación","Rubro1_Equipamiento para el acondicionamiento, distribución y filtrado de fluidos",...,"Rubro1_Servicios de perforación de minería, petróleo y gas",Rubro1_Servicios de producción y fabricación industrial,"Rubro1_Servicios de transporte, almacenaje y correo","Rubro1_Servicios editoriales, de diseño, publicidad, gráficos y artistas","Rubro1_Servicios financieros, pensiones y seguros",Rubro1_Servicios medioambientales,"Rubro1_Servicios profesionales, administrativos y consultorías de gestión empresarial","Rubro1_Tecnologías de la información, telecomunicaciones y radiodifusión",Rubro1_Vehículos y equipamiento en general,Descripcion raiz limpia
id_producto,,,,,,,,,,,,,,,,,,,,,
2792-88-LE21-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,cam retinal midriat especif bas tecnic
1704-52-LP21-1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,servici mudanz enser propi hospital martin qui...
1003-20-L121-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,leñ sec certific
4236-21-L121-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,marcador blanc metal
4236-21-L121-2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,plom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058141-35-LQ21-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,alteplas caj
1058141-35-LQ21-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tenecteplas
1173455-12-LE21-41,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,manguer transparent nivel


In [57]:
#Creamos nuestra bolsa de palabras

count = CountVectorizer()
descripcion = np.array(df1['Descripcion limpia'])
bag = count.fit_transform(descripcion)

In [ ]:
distribucion=nltk.FreqDist(count.vocabulary_) 
lista_frecuencias=distribucion.most_common() #Para ordenar la distribución de la frecuencia de palabras
print(lista_frecuencias)

In [62]:
len(n_palabras)

19535

In [61]:
#Observamos las palabras presentadas en las descripciones limpias
n_palabras= count.vocabulary_
n_palabras

{'cam': 2929,
 'retinal': 15773,
 'midriat': 11890,
 'especif': 6851,
 'bas': 2016,
 'tecnic': 17575,
 'servici': 16535,
 'mudanz': 12299,
 'enser': 6591,
 'propi': 14843,
 'hospital': 9129,
 'martin': 11382,
 'quillot': 15160,
 'ofert': 12991,
 'segun': 16383,
 'anex': 969,
 'leñ': 10668,
 'sec': 16343,
 'certific': 3485,
 'marcador': 11321,
 'blanc': 2390,
 'metal': 11698,
 'plom': 14233,
 'azadon': 1821,
 'mang': 11247,
 'escobillon': 6777,
 'ase': 1471,
 'municipal': 12405,
 'caj': 2841,
 'lapiz': 10447,
 'carpinter': 3193,
 'hoj': 9063,
 'marcosierr': 11332,
 'cint': 3762,
 'papel': 13485,
 'equivalent': 6704,
 'masking': 11401,
 'tap': 17503,
 'pegot': 13723,
 'ancho': 927,
 'carpet': 3190,
 'carton': 3227,
 'pigment': 14041,
 'ofici': 12999,
 'color': 4192,
 'azul': 1840,
 'plastific': 14196,
 'accoclip': 99,
 'archiv': 1310,
 'tamañ': 17484,
 'auc': 1621,
 'cart': 3218,
 'plumon': 14244,
 'permanent': 13887,
 'punt': 15038,
 'redond': 15465,
 'roj': 15968,
 'negr': 12575,
 'ver

In [64]:
from sklearn.feature_extraction.text import TfidfTransformer
#creamos nuestra matriz TF - IDF con el df1
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(bag).toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [65]:
tfidf = TfidfTransformer(norm=None)
tfidf = tfidf.fit_transform(bag).toarray()
tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
vectorizador = TfidfVectorizer(max_features)
 
#Crear una matriz dispersa
cantidades = count.fit_transform(descripcion)
valores = vectorizador.fit_transform(descripcion)
 
print ("--- Palabras ---")
print(vectorizador.get_feature_names())
print ("... Vector de cantidades ...")
print (cantidades)



--- Palabras ---
['adjunt', 'bas', 'caj', 'certific', 'comprim', 'convenient', 'eficaci', 'fabric', 'men', 'mes', 'preci', 'presentacion', 'public', 'punt', 'segun', 'servici', 'simil', 'tecnic', 'vent', 'vist']
... Vector de cantidades ...
  (0, 2929)	1
  (0, 15773)	1
  (0, 11890)	1
  (0, 6851)	1
  (0, 2016)	1
  (0, 17575)	1
  (1, 16535)	1
  (1, 12299)	1
  (1, 6591)	1
  (1, 14843)	1
  (1, 9129)	1
  (1, 11382)	1
  (1, 15160)	1
  (1, 12991)	1
  (1, 16383)	1
  (1, 969)	1
  (2, 10668)	1
  (2, 16343)	1
  (2, 3485)	1
  (3, 11321)	1
  (3, 2390)	1
  (3, 11698)	1
  (4, 14233)	1
  (5, 1821)	1
  (5, 11247)	1
  :	:
  (54841, 11758)	1
  (54841, 14038)	1
  (54841, 4098)	1
  (54842, 14371)	1
  (54842, 13077)	1
  (54842, 5790)	1
  (54842, 5825)	1
  (54843, 9888)	1
  (54843, 14371)	1
  (54843, 10803)	1
  (54843, 17865)	1
  (54844, 2841)	1
  (54844, 733)	1
  (54845, 17655)	1
  (54846, 12760)	1
  (54846, 18186)	1
  (54846, 11250)	1
  (54847, 3762)	1
  (54847, 276)	1
  (54848, 13037)	1
  (54848, 13160)	1

In [68]:
print ("¨¨¨ Vector de valores ¨¨")
print (valores)

¨¨¨ Vector de valores ¨¨
  (0, 17)	0.6846135730104439
  (0, 1)	0.7289062049741885
  (1, 14)	0.6544606283711224
  (1, 15)	0.7560960824604738
  (2, 3)	1.0
  (7, 2)	1.0
  (14, 13)	0.6795855687976756
  (14, 2)	0.7335962477289122
  (15, 13)	0.6795855687976756
  (15, 2)	0.7335962477289122
  (16, 13)	0.6795855687976756
  (16, 2)	0.7335962477289122
  (17, 13)	0.6795855687976756
  (17, 2)	0.7335962477289122
  (22, 13)	0.6795855687976756
  (22, 2)	0.7335962477289122
  (23, 13)	0.6795855687976756
  (23, 2)	0.7335962477289122
  (26, 0)	0.5891828075735706
  (26, 14)	0.5742043730678881
  (26, 17)	0.5684654406465167
  (28, 14)	0.7106491837890527
  (28, 17)	0.7035465425826162
  (29, 14)	0.7106491837890527
  (29, 17)	0.7035465425826162
  :	:
  (54817, 16)	1.0
  (54818, 4)	0.6471601832268454
  (54818, 2)	0.7623540497995639
  (54819, 4)	0.6471601832268454
  (54819, 2)	0.7623540497995639
  (54820, 4)	0.6471601832268454
  (54820, 2)	0.7623540497995639
  (54821, 4)	0.6471601832268454
  (54821, 2)	0.76235404